In [2]:
import argparse
import cv2
import os


In [2]:
DATA = "../data/raw/GX010013.MP4"

In [3]:
# Check video dimensions. Width: 5312, Height: 2988
cap = cv2.VideoCapture("../data/preprocessed/GX010014_cropped.MP4")
ret, frame = cap.read()
if ret:
    h, w, _ = frame.shape
    print(f"Width: {w}, Height: {h}")
cap.release()



Width: 4100, Height: 1300


In [ ]:
# Crop and visualize video based on ROI
import cv2

cap = cv2.VideoCapture("../data/raw/GX010014.MP4")
ret, frame = cap.read()
cap.release()

if ret:
    x, y, w, h = 0, 0, 4100, 1300
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
    resized = cv2.resize(frame, (1280, 720))  # Para visualización
    cv2.imshow("ROI Preview", resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Could not read first frame.")

In [ ]:
# Crop video function
import cv2

def crop_video(input_path, x, y, w, h, output_path=None):
    if output_path is None:
        output_path = input_path.rsplit('.', 1)[0] + "_cropped.MP4"

    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        raise ValueError(f"Cannot open video: {input_path}")

    # Get original properties
    orig_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    orig_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Validate ROI
    if x < 0 or y < 0 or x + w > orig_w or y + h > orig_h:
        raise ValueError(f"ROI ({x}, {y}, {w}, {h}) out of bounds for video size ({orig_w}, {orig_h})")

    # Use H.264 codec if available (more compatible)
    fourcc = cv2.VideoWriter_fourcc(*"avc1")  # H.264
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    if not out.isOpened():
        # Fallback to mp4v if avc1 fails
        print("Warning: 'avc1' codec failed. Trying 'mp4v'.")
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))
        if not out.isOpened():
            raise RuntimeError("Failed to open VideoWriter with any codec.")

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cropped = frame[y:y+h, x:x+w]
        out.write(cropped)
        frame_count += 1

    cap.release()
    out.release()
    print(f"Cropped video saved: {output_path} ({frame_count} frames)")


X, Y, W, H = 0, 0, 4100, 1300
crop_video("../data/raw/GX010014.MP4", X, Y, W, H)

[ERROR:0@2815.389] global cap_ffmpeg_impl.hpp:3133 open Could not find encoder for codec_id=27, error: Encoder not found
[ERROR:0@2815.390] global cap_ffmpeg_impl.hpp:3211 open VIDEOIO/FFMPEG: Failed to initialize VideoWriter


Cropped video saved: ../data/raw/GX010014_cropped.mp4 (1965 frames)


In [ ]:
# # Crop video function
# def crop_video(input_path, x, y, w, h, output_path=None):
#     if output_path is None:
#         output_path = input_path[:-4] + "_cropped.MP4"

#     cap = cv2.VideoCapture(input_path)
#     fps = int(cap.get(cv2.CAP_PROP_FPS))
#     fourcc = cv2.VideoWriter_fourcc(*"mp4v")
#     out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         cropped = frame[y:y+h, x:x+w]
#         out.write(cropped)
#     cap.release()
#     out.release()

# # ROI (Region of Interest) coordinates. Default ROI values based on manual selection
# # X, Y, W, H = 0, 0, 4321, 1722 
# X, Y, W, H = 1677, 271, 2134, 1198
# crop_video("../data/raw/GX010014.MP4", X, Y, W, H)

In [ ]:
# # Extract frames function
# def extract_frames(video_path, output_dir, interval=10):
#     os.makedirs(output_dir, exist_ok=True)
#     cap = cv2.VideoCapture(video_path)
#     frame_id = 0
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break
#         if frame_id % interval == 0:  # Saves 1 frame every N-interval frames
#             cv2.imwrite(f"{output_dir}/{video_path[18:-12]}_F{frame_id:06d}.jpg", frame)
#         frame_id += 1
#     cap.release()


# # Extract frames from cropped video
# extract_frames(
#     video_path="../data/preprocessed/GX010013_cropped.MP4",
#     output_dir="../data/frames",
#     interval=30
# )

In [ ]:
# Interactive frame extractor with navigation
import cv2
import os

video_path = "../data/preprocessed/GX010014_cropped.MP4"
output_dir = "../data/interactive_frames"
scale_percent = 40 # Resize percentage

os.makedirs(output_dir, exist_ok=True)
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise IOError("Cannot open video file")

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
current_frame = 0

def show_frame(frame_idx):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    if not ret:
        return None
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    resized = cv2.resize(frame, (width, height))
    cv2.imshow("Frame", resized)
    return frame

frame = show_frame(current_frame)

while True:
    if frame is None:
        break

    key = cv2.waitKey(0) & 0xFF

    if key == ord('q'):
        break

    elif key == ord('s'):
        cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        ret, original_frame = cap.read()
        if ret:
            output_path = os.path.join(output_dir, f"{video_path[18:-12]}_F{current_frame:06d}.jpg")
            cv2.imwrite(output_path, original_frame) # Saves the original frame, not the resized one
            print(f"Saved {output_path}")

    elif key == 81 or key == 2:  # Flecha izquierda (código varía por sistema)
        current_frame = max(0, current_frame - 1)
        frame = show_frame(current_frame)

    elif key == 83 or key == 3:  # Flecha derecha
        current_frame = min(total_frames - 1, current_frame + 1)
        frame = show_frame(current_frame)

    else:
        continue

cap.release()
cv2.destroyAllWindows()

Saved ../data/interactive_frames/GX010014_F000037.jpg
Saved ../data/interactive_frames/GX010014_F000043.jpg
Saved ../data/interactive_frames/GX010014_F000047.jpg
Saved ../data/interactive_frames/GX010014_F000088.jpg
Saved ../data/interactive_frames/GX010014_F000101.jpg
Saved ../data/interactive_frames/GX010014_F000104.jpg
Saved ../data/interactive_frames/GX010014_F000110.jpg
Saved ../data/interactive_frames/GX010014_F000138.jpg
Saved ../data/interactive_frames/GX010014_F000199.jpg
Saved ../data/interactive_frames/GX010014_F000206.jpg
Saved ../data/interactive_frames/GX010014_F000222.jpg
Saved ../data/interactive_frames/GX010014_F000255.jpg
Saved ../data/interactive_frames/GX010014_F000284.jpg
Saved ../data/interactive_frames/GX010014_F000310.jpg
Saved ../data/interactive_frames/GX010014_F000321.jpg
Saved ../data/interactive_frames/GX010014_F000341.jpg
Saved ../data/interactive_frames/GX010014_F000356.jpg
Saved ../data/interactive_frames/GX010014_F000363.jpg
Saved ../data/interactive_fr